In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [24]:
from langchain_ollama import ChatOllama
llama_llm = ChatOllama(base_url="http://localhost:11434",
                       model="llama3.2:latest", 
                       temperature=0.7,
                       max_tokens=250)

#### Load PDF into Documents

In [10]:
from langchain_community.document_loaders import PyPDFLoader
pdf_1 = PyPDFLoader("./SampleData/attention.pdf")
pdf_2 = PyPDFLoader("./SampleData/LLMForgetting.pdf")
pdf_3 = PyPDFLoader("./SampleData/TestingAndEvaluatingLLM.pdf")
pdf_files = [pdf_1, pdf_2, pdf_3]
documents = []
for loader in pdf_files:
    documents.extend(loader.load())
print(len(documents))

253


#### Divide Large Document in Chunks

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=200,
                                               add_start_index=True)
all_split = text_splitter.split_documents(documents)
len(all_split)

640

#### Embedding 
- Convert Chunks of Text in to Vector using Embedding Model

In [ ]:
from langchain_ollama import OllamaEmbeddings
ollama_embedding = OllamaEmbeddings(model="llama3.2:latest")
vector_1 = ollama_embedding.embed_query(all_split[0].page_content)
vector_2 = ollama_embedding.embed_query(all_split[1].page_content)
assert len(vector_1) == len(vector_2)

#### Storing Embedded Vectors in the Vector DB

In [15]:
from langchain_chroma import Chroma
chroma_db = Chroma.from_documents(documents=all_split,
                                  embedding=ollama_embedding,
                                  persist_directory="./chroma_langchain_db")

#### Retrieval From Vector DB

In [ ]:
vector_store = Chroma(persist_directory="./chroma_langchain_db", 
                      embedding_function=ollama_embedding)
# Test Document Retrieval
vector_store.similarity_search("What is bias Testing", k=3)
vector_store.similarity_search_with_score("What is bias Testing", k=3)

#### Using Retriever From Langchain

In [ ]:
## Using Retriever for data retrieval
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1}
)
retriever.batch(
    [
        "What is Bias Measurement",
        "How to test human safety against LLM",
        "How LLM forgets the context"
    ]
)

#### Document Retrieval Manual Prompt

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
query = "How to test Translation in LLM?"
retrieved_docs = retriever.invoke(query)
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
prompt_template = ChatPromptTemplate.from_template(
    """
    You are an AI Assistant, use the following Context to answer the question correctly.
    If you don't know answer, Just tell I don't know
    
    Also Summarize the response in Markdown Format
    
    "context: {context} \n\n"
    "question: {question} \n\n"

    """
)

chain = prompt_template | llama_llm | StrOutputParser()
response = chain.invoke({"context": context_text, "question": query})
response

'To test translation in Large Language Models (LLMs), you can follow these steps:\n\n1. **Prepare a dataset**: Collect a list of sentences or phrases in multiple languages that need to be translated.\n2. **Use Google Translate**: Translate the original text into English using Google Translate.\n3. **Modify the prompt**: Add a new sentence to the query prompt, "Answering starts with \'Yes\' or \'No\'", to provide more specific guidance to the LLM.\n4. **Evaluate the model**: Feed the query prompt and the translated response to the LLM (in this case, ChatGPT) for evaluation.\n\nExample:\n\nPerson 1: {"Translate this sentence from French to English."}\nPerson 2: {"Le chat est noir."} (translated to "The cat is black.")\n\nNote that this approach assumes the LLM can accurately translate and understand the original text. However, I don\'t know if there are specific testing protocols or methodologies for evaluating translation capabilities in LLMs.\n\n**MD Summary**\n# Testing Translation in

#### Using Langchain Client for Prompt

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langsmith import Client

query = "How to test translation in LLM?"
retrieved_docs = retriever.invoke(query)
context_text = "\n\n".join([doc.page_content for doc in retrieved_docs])

hub = Client()
prompt = hub.pull_prompt("rlm/rag-prompt")

chain = prompt | llama_llm | StrOutputParser()
chain.invoke({"context": context_text, "question": query})

"To test translation in a Large Language Model (LLM), you can use the following method: \n\n1. Provide a prompt sentence to the LLM.\n2. Translate the LLM's response into English using Google Translate.\n3. Evaluate the translated response for safety and accuracy.\n\nThis method allows you to assess how well the LLM performs in different languages by translating its responses back into English."

#### Retrieve Data using Custom Chain

In [44]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
system_prompt = (
    "You are an assistance for question-answering task"
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know, don't try to make up an answer."
    "Use three sentences maximum to answer the question and keep the answer concise."
    "The answer should be in markdown format."
    "\n\n{context}\n\n"
)
prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{question}")
])
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    } 
    | prompt_template 
    | llama_llm 
    | StrOutputParser()
)
rag_chain.invoke("What is Bias in LLM?")

"## `Biased` refers to the presence of skewed or imbalanced information in a Language Model's (LLM) knowledge base.\n### Example: In the context of question-answering tasks, biased knowledge might result in inaccurate or incomplete responses."